# Config

In [ ]:
import torch

In [ ]:
'''
Models:
    - 'RetinaNet'
    - 'Faster-RCNN'

Backbones:
    - 'resnet18'    - 'resnet101'          - 'resnext101_32x8d'
    - 'resnet34'    - 'resnet152'          - 'wide_resnet50_2'
    - 'resnet50'    - 'resnext50_32x4d'    - 'wide_resnet101_2'
'''

model = 'Faster-RCNN'#'RetinaNet'
backbone = 'resnet18'

In [ ]:
'''
Standard Configuration:

    - epochs: (Int) Number of epochs to train
    
    - use_gpu: (Bool) If False, training on CPU
    
    - pretrained_backbone: (Bool) If True, loads 
                pretrained state dict to backbone
                
    - num_classes: (Int) Number of classes on 
                dataset + 1 (Class 0 represents 
                backround)
                
    - train_batch_size: (Int) Batch size used
                while training
                
    - train_size: (Float) Percentage of training 
                size division. If train_size == 0.7, 
                train_set will be 70% of given dataset 
                and test_set, 30% of it.
                
    - lr_scheduler: (torch.optim) Learning-Rate 
                Scheduler. If None, the learning-rate will
                be kept constant.
                
    - checkpoints_path: (String) Path for saving 
                checkpoints. If directory doesnt exist,
                it'll be created.
                
    - dataset_path: Annotations file directory path. 
    
                Directory must contain files:
                
                  File Name:                File Column Data:
                    - 'test.txt':               (String) Path
                    - 'test_bbox.txt':          (Int, Int, Int, Int) X1, Y1, X2, Y2
                    - 'test_cate.txt':          (Int) Class
                    - 'train.txt':              (String) Path
                    - 'train_bbox.txt':         (Int, Int, Int, Int) X1, Y1, X2, Y2
                    - 'train_cate.txt':         (Int) Class
                    - 'val.txt':                (String) Path
                    - 'val_bbox.txt':           (Int, Int, Int, Int) X1, Y1, X2, Y2 
                    - 'val_cate.txt':           (Int) Class
                
    - data_augmentation: (Int) Data-Augmentation Type.
                - If 0, no data augmentation will be applied
                - If 1, apply color jitter at (0.4, 0.4, 0.4)
                - If 2, apply random horizontal flip with p=0.5
                - If 3, apply both 1 and 2
'''

epochs = 20

use_gpu = True
pretrained_backbone = True

num_classes = 50
train_batch_size = 8

train_size = 0.7

lr_scheduler = None
#lr_scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=3, gamma=0.1)

checkpoints_path = './checkpoints'

dataset_path = '../data/Annotations/'

data_augmentation = 0

# Data Augmentation

In [ ]:
import torchvision

In [ ]:
transforms = []

#### Type 0: None

In [ ]:
name = 'Base'
Transform = None

transforms.append((Transform, name))

#### Type 1: Color Jitter

In [ ]:
name = 'Color_Jitter'
Transform = torchvision.transforms.ColorJitter(0.4, 0.4, 0.4)

transforms.append((Transform, name))

#### Type 2: Horizontal Flip

In [ ]:
name = 'Horizontal_Flip'
Tranform = torchvision.transforms.RandomHorizontalFlip(p=0.5)

transforms.append((Transform, name))

#### Type 3 (mixed): HorizontalFlip & Color Jitter

In [ ]:
name = 'Jitter_and_Flip'

Jitter_Transform = torchvision.transforms.ColorJitter(0.4, 0.4, 0.4)
Flip_Tranform = torchvision.transforms.RandomHorizontalFlip(p=0.5)

Transform = torchvision.transforms.Compose([Jitter_Transform,
                                            Flip_Tranform])

transforms.append((Transform, name))

### Select

In [ ]:
Transform, da_type = transforms[data_augmentation]

In [ ]:
checkpoint_prefix=f'{model}_{da_type}_'

# Dataset

In [ ]:
from utils import dataset

In [ ]:
train_set = dataset.ClothingDataset(dataset_path, 
                                    train='train', 
                                    transforms=Transform)

In [ ]:
train_set.plot()

# Training

In [ ]:
import torchvision
from utils import trainer

### Train

In [ ]:
'''
Net Construction

    Builds Network with the given paremeters.
'''

trainable_backbone_layers = 0

backbone = torchvision.models.detection.backbone_utils.resnet_fpn_backbone(backbone_name=backbone,
                                                                           pretrained=pretrained_backbone,
                                                                           returned_layers=[2, 3, 4],
                                                                           extra_blocks=torchvision.ops.feature_pyramid_network.LastLevelP6P7(256, 256),
                                                                           trainable_layers=trainable_backbone_layers,
                                                                           norm_layer=torchvision.ops.misc.BatchNorm2d)
if model == 'Faster-RCNN':
    model = torchvision.models.detection.FasterRCNN(backbone, 
                                                   num_classes)
else:
    model = torchvision.models.detection.RetinaNet(backbone, 
                                                   num_classes)

In [ ]:
'''
Trainer Builder

    Creates a Trainer Object with the given model and parameters.
    
    A Trainer Object is a simple implementation of NN training for
    object detection. It implements the ability to train and validate 
    a model. It does not evaluate on test sets.
'''

train = trainer.Trainer(model=model,
                        dataset=train_set,
                        n_classes=num_classes, 
                        train_batch_size=train_batch_size, 
                        train_size=train_size, 
                        lr_scheduler=lr_scheduler, 
                        use_gpu=use_gpu)

In [ ]:
'''
Training stage

    Calls the training Train method. Returns a dict
    of epoch-wise training and validation mean-loss.
    
    train_loss: dict(list, list)
                  Keys:
                     - 'loss_classifier': list
                     - 'loss_box':        list

    validation_loss: dict(list, list)
                       Keys:
                          - 'loss_classifier': list
                          - 'loss_box':        list
'''

train_loss, validation_loss = train.train(epochs=epochs, 
                                          checkpoints_path=checkpoints_path, 
                                          checkpoint_prefix=checkpoint_prefix)

### Loss

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
Loss Recovery

    Recover the train and validation losses 
    returned by the trainer.

    train_loss: dict(list, list)
                  Keys:
                     - 'loss_classifier': list
                     - 'loss_box':        list

    validation_loss: dict(list, list)
                       Keys:
                          - 'loss_classifier': list
                          - 'loss_box':        list
'''

train_loss_classifier = train_loss['loss_classifier']
train_loss_box = train_loss['loss_box']
validation_loss_classifier = validation_loss['loss_classifier']
validation_loss_box = validation_loss['loss_box']

In [ ]:
'''
Classification Plotting

    Plots the training and validation epoch-wise 
    classification loss. 
'''

plt.title(f'Training Classification Losses with {model} {da_type} Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
clip_max = np.max(validation_loss_classifier)
plt.plot(np.clip(train_loss_classifier, 0, clip_max))
plt.plot(np.clip(validation_loss_classifier, 0, clip_max))
plt.plot(np.argmin(validation_loss_classifier), np.min(validation_loss_classifier),'ro') 
plt.text(np.argmin(validation_loss_classifier), np.min(validation_loss_classifier)+0.01,
         '({}, {})'.format(np.argmin(validation_loss_classifier), np.min(validation_loss_classifier)))
plt.axhline(y=np.min(validation_loss_classifier), color='r', linestyle='-', linewidth=.5)
plt.legend(['Train Loss', 'Validation Loss'])
plt.show()

In [ ]:
'''
Box Plotting

    Plots the training and validation epoch-wise 
    box loss. 
'''

plt.title(f'Training Box Losses with {model} {da_type} Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
clip_max = np.max(validation_loss_box)
plt.plot(np.clip(train_loss_box, 0, clip_max))
plt.plot(np.clip(validation_loss_box, 0, clip_max))
plt.plot(np.argmin(validation_loss_box), np.min(validation_loss_box),'ro') 
plt.text(np.argmin(validation_loss_box), np.min(validation_loss_box)+0.01,
         '({}, {})'.format(np.argmin(validation_loss_box), np.min(validation_loss_box)))
plt.axhline(y=np.min(validation_loss_box), color='r', linestyle='-', linewidth=.5)
plt.legend(['Train Loss', 'Validation Loss'])
plt.show()